In [1]:
import tensorflow as tf
import tensorflow.keras as keras

2023-10-15 21:56:15.745420: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-15 21:56:15.745455: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-15 21:56:15.745482: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-15 21:56:15.751615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
num_classes = 10
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
image_vector_size = 28*28
x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [3]:
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, input_shape = (784,)),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Dense(10)
])

2023-10-15 21:56:17.020181: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 21:56:17.025054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 21:56:17.025311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
optimizer = tf.optimizers.Adam()
loss_history = []
dataset = tf.data.Dataset.from_tensor_slices((
    tf.cast(x_train/255.0, tf.float32),
    tf.cast(y_train, tf.int64))
)
dataset = dataset.shuffle(60000).batch(64)

2023-10-15 21:56:17.295216: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 376320000 exceeds 10% of free system memory.


In [8]:
%%time
with tf.device("/cpu:0"):
    for i in range(10):
        print("\nEpoch: ", i)
        for (batch, (images, labels)) in enumerate(dataset.take(60000)):
            if batch % 100 == 0:
                print(".", end="")
            with tf.GradientTape() as tape:
                with tf.device('/device:GPU:0'):
                    logits = mnist_model(images, training = True)
                with tf.device("/cpu:0"):
                    tgmax = tf.argmax(labels, axis = 1, output_type = tf.int64)
                with tf.device('/device:GPU:0'):
                    loss_value = tf.compat.v1.losses.sparse_softmax_cross_entropy(
                        tgmax,
                        logits
                    )
                    loss_history.append(loss_value.numpy())
                    grads = tape.gradient(loss_value, mnist_model.variables)
                    optimizer.apply_gradients(zip(grads, mnist_model.variables), global_step = tf.compat.v1.train.get_or_create_global_step())



Epoch:  0
..........
Epoch:  1
..........
Epoch:  2
..........
Epoch:  3
..........
Epoch:  4
..........
Epoch:  5
..........
Epoch:  6
..........
Epoch:  7
..........
Epoch:  8
..........
Epoch:  9
..........CPU times: user 1min 9s, sys: 1.57 s, total: 1min 11s
Wall time: 1min 10s


In [7]:
%%time
for i in range(10):
    print("\nEpoch: ", i)
    for (batch, (images, labels)) in enumerate(dataset.take(60000)):
        if batch % 100 == 0:
            print(".", end="")
        with tf.GradientTape() as tape:
            logits = mnist_model(images, training = True)
            tgmax = tf.argmax(labels, axis = 1, output_type = tf.int64)
            loss_value = tf.compat.v1.losses.sparse_softmax_cross_entropy(
                tgmax,
                logits
            )
            loss_history.append(loss_value.numpy())
            grads = tape.gradient(loss_value, mnist_model.variables)
            optimizer.apply_gradients(zip(grads, mnist_model.variables), global_step = tf.compat.v1.train.get_or_create_global_step())



Epoch:  0
..........
Epoch:  1
..........
Epoch:  2
..........
Epoch:  3
..........
Epoch:  4
..........
Epoch:  5
..........
Epoch:  6
..........
Epoch:  7
..........
Epoch:  8
..........
Epoch:  9
..........CPU times: user 1min 11s, sys: 1.68 s, total: 1min 13s
Wall time: 1min 11s


In [ ]:
probs = tf.nn.softmax(mnist_model(x_train))
a = tf.reduce_mean(
    tf.cast(
        tf.equal(
            tf.argmax(probs, axis = 1), 
            tf.argmax(y_train, axis = 1)
        ),
        tf.float32)
    )
print(a)

tf.Tensor(0.88155, shape=(), dtype=float32)
